In [ ]:
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

### Remove some months based on missing rate

remove data after 2021-06-30

In [ ]:
price_1min = pd.read_csv('./mid_price_1min.csv')

In [ ]:
price_1min

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = price_1min
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

df['missing_rate'] = df.iloc[:, 2:].isnull().mean(axis=1)  

df['year'] = df['datetime'].dt.year
yearly_missing_rate = df.groupby('year')['missing_rate'].mean()

flierprops = dict(marker='o', markersize=3, linestyle='none', markeredgewidth=0.5, markeredgecolor='black')
plt.figure(figsize=(5, 4))
bp = plt.boxplot([df[df['year'] == year]['missing_rate'] for year in df['year'].unique()],
            labels=df['year'].unique(), patch_artist=True, flierprops=flierprops)

# '#116DA9'
# '#B03C2B'
for box in bp['boxes']:
    box.set_facecolor('#B03C2B')
    box.set_alpha(0.9)

plt.xlabel('Year', fontsize=14)
plt.ylabel('Missing Rate', fontsize=14)
plt.xticks(fontsize=12, rotation=90)
plt.yticks(fontsize=12)
# plt.title('Box Plot of Yearly Missing Rate')
plt.ylim([-0.05, 1.05])

# plt.show()
plt.savefig(f'./yearly_missing_rate.pdf', bbox_inches='tight')
plt.close()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = price_1min
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month

df_2021 = df[df['year'] == 2021].copy()

df_2021['missing_rate'] = df_2021.iloc[:, 2:-3].isnull().mean(axis=1)

flierprops = dict(marker='o', markersize=3, linestyle='none', markeredgewidth=0.5, markeredgecolor='black')
plt.figure(figsize=(5, 4.3))
bp = plt.boxplot([df_2021[df_2021['month'] == month]['missing_rate'] for month in range(1, 13)],
            labels=[f'{month}' for month in range(1, 13)], patch_artist=True, flierprops=flierprops)

for box in bp['boxes']:
    box.set_facecolor('#B03C2B')
    box.set_alpha(0.9)

plt.xlabel('Month', fontsize=14)
plt.ylabel('Missing Rate', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim([-0.05, 1.05])

# plt.title('Box Plot of Monthly Missing Rate in 2021')
# plt.show()
plt.savefig(f'./monthly_missing_rate_2021.pdf', bbox_inches='tight')
plt.close()


### Remove some tickers

#### avg_daily_volume

remove bottom 40% (from 516 to 317)

In [ ]:
def load_and_calculate_avg_volume(csv_file):
    df = pd.read_csv(csv_file)
    
    # Outlier condition based on ask_1 and bid_1
    outlier_condition = (df['ask_1'] <= 0) | (df['bid_1'] <= 0) | ((df['ask_1'] / df['bid_1']) > 2)
    
    # Calculate the average volume for each minute
    df['avg_volume'] = np.where(outlier_condition, 0, (df['ask_size_1'] + df[' bid_size_1']) / 2)
    
    # Sum up the volume for the entire day
    total_daily_volume = df['avg_volume'].sum()
    
    return total_daily_volume

In [ ]:
def process_stock_folder(stock_folder, root_dir):
    stock_folder_path = os.path.join(root_dir, stock_folder)
    if os.path.isdir(stock_folder_path):
        try:
            stock_name = stock_folder.split('_')[6]
        except Exception as e:
            print(f"File path: {stock_folder_path}. Error: {e}")
            return None
        
        daily_volumes = []
        for csv_file in os.listdir(stock_folder_path):
            csv_file_path = os.path.join(stock_folder_path, csv_file)
            if csv_file_path.endswith('.csv'):
                try:
                    daily_volume = load_and_calculate_avg_volume(csv_file_path)
                    daily_volumes.append(daily_volume)
                except Exception as e:
                    print(f"File path: {csv_file_path}. Error: {e}")
        
        if daily_volumes:
            avg_daily_volume = np.mean(daily_volumes)
            return {'stock': stock_name, 'avg_daily_volume': avg_daily_volume}
    return None

def compile_volume_data(root_dir):
    stock_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
    
    compiled_data = Parallel(n_jobs=-1)(delayed(process_stock_folder)(folder, root_dir) for folder in tqdm(stock_folders))
    
    # Filter out None values
    compiled_data = [data for data in compiled_data if data is not None]
    
    # Convert to DataFrame
    result_df = pd.DataFrame(compiled_data)
    
    return result_df

root_directory = '../LOB_516_Minutely_2007_2021/data_by_stocks'
volume_data_df = compile_volume_data(root_directory)
volume_data_df.head()


In [ ]:
# volume_data_df.to_csv('./avg_daily_volume.csv', index=False)

In [ ]:
volume_data_df = pd.read_csv('avg_daily_volume.csv')

In [ ]:
volume_data_df.avg_daily_volume.describe([0.1, 0.2, .25, .3, .4, .5, .75])

In [ ]:
volume_data_df.avg_daily_volume > 1e+05

#### Missing rate

remove tickers with missing rate > 5% (num of tickers reduced from 516 to 408)

In [ ]:
price_1min = pd.read_csv('./mid_price_1min.csv')

In [ ]:
df = price_1min
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

cutoff_date = pd.Timestamp('2021-07-01')
df = df[df['datetime'] < cutoff_date]
df = df.drop(columns=['datetime'])

In [ ]:
df.isna().mean().sort_values().describe([.25, .5, .75, 0.8, 0.81, 0.82, 0.83, 0.84, .85, .9])

In [ ]:
df.isna().mean()[2:] <= 0.05

### process log_returns: remove high missing rate months and remove some tickers

In [ ]:
log_returns_1min = pd.read_csv('./log_returns_1min.csv')

In [ ]:
# volume
high_volume_stocks = volume_data_df[volume_data_df['avg_daily_volume'] > 1e+05]['stock']
high_volume_stocks

In [ ]:
# missing rate
valid_stocks = df.isna().mean()[2:] <= 0.05
valid_stocks = valid_stocks[valid_stocks].index
valid_stocks

In [ ]:
selected_stocks = set(high_volume_stocks).intersection(valid_stocks)

In [ ]:
log_returns_1min_filtered = log_returns_1min[['date', 'time'] + list(selected_stocks)]

In [ ]:
log_returns_1min_filtered = log_returns_1min_filtered.copy()
log_returns_1min_filtered['date'] = pd.to_datetime(log_returns_1min_filtered['date'])

log_returns_1min_filtered = log_returns_1min_filtered[log_returns_1min_filtered['date'] < '2021-07-01']

In [ ]:
log_returns_1min_filtered

In [ ]:
log_returns_1min_filtered.to_csv('./log_returns_1min_252.csv', index=False)